In [198]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import xgboost as xgb
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
import pickle

In [150]:
flights = pd.read_csv('data/flights_cleaned.csv').drop(columns=['Unnamed: 0'])

In [151]:
def time_row(row):
    row_int = int(row)
    row_str = str(row_int)
    len_row = len(row_str)
    if len_row == 1:
        minute = '0' + row_str
        hour = '00'
        row = hour + ':' + minute
    if len_row == 2:
        minute =  row_str[0] + row_str[1]
        hour = '00'
        row = hour + ':' + minute
    if len_row == 3:
        minute = row_str[1]+row_str[2]
        hour = '0' + row_str[0]
        row = hour + ':' + minute
    elif len_row == 4:
        minute = row_str[2] + row_str[3]
        hour = row_str[0] + row_str[1]
        row = hour + ':' + minute
        if row == '24:00':
            row = '23:59'
    return str(row)

In [152]:
flights['crs_dep_time'] = flights['crs_dep_time'].apply(time_row)
flights['dep_time'] = flights['dep_time'].apply(time_row)
flights['wheels_off'] = flights['wheels_off'].apply(time_row)
flights['wheels_on'] = flights['wheels_on'].apply(time_row)
flights['crs_arr_time'] = flights['crs_arr_time'].apply(time_row)
flights['arr_time'] = flights['arr_time'].apply(time_row)

In [153]:
#lets create year, month and day
flights['fl_date'] = pd.to_datetime(flights['fl_date'],format='%Y-%m-%d')
flights['year'] = pd.DatetimeIndex(flights['fl_date']).year
flights['month'] = pd.DatetimeIndex(flights['fl_date']).month
flights['day'] = pd.DatetimeIndex(flights['fl_date']).day

In [154]:
#lets change the time columns format to time
flights['crs_dep_time'] = pd.to_datetime(flights['crs_dep_time'],format="%H:%M").dt.time
flights['dep_time'] = pd.to_datetime(flights['dep_time'],format="%H:%M").dt.time
flights['wheels_off'] = pd.to_datetime(flights['wheels_off'],format="%H:%M").dt.time
flights['wheels_on'] = pd.to_datetime(flights['wheels_on'],format="%H:%M").dt.time
flights['crs_arr_time'] = pd.to_datetime(flights['crs_arr_time'],format='%H:%M').dt.time
flights['arr_time'] = pd.to_datetime(flights['arr_time'],format="%H:%M").dt.time

In [155]:
#dropping the large delays, they seem to be outlier
flights = flights[flights['dep_delay'] <= 15]
flights['crs_arr_hour'] = flights['crs_arr_time'].apply(lambda x:x.hour)

In [157]:
top_dest = flights.groupby('dest')['taxi_in'].count().sort_values(ascending=False).head(12)

In [158]:
list_dest = []
for i in range(0,12):
    list_dest.append(top_dest.index[i])
flights=flights.loc[flights['dest'].isin(list_dest)]

In [159]:
#arrival delay filter based on hour of the day
flights = flights.loc[(flights['crs_arr_hour'] >=6) & flights['crs_arr_hour'] <=18]

In [161]:
flights = flights[['dest','mkt_unique_carrier','crs_arr_hour','air_time',
                   'taxi_in','crs_elapsed_time','month',
                  'arr_delay']]
numeric =flights[['air_time','taxi_in','crs_elapsed_time','arr_delay']]
dummies = pd.get_dummies(flights[['dest','mkt_unique_carrier','crs_arr_hour','month']],drop_first=True)


In [164]:
all_in_one = pd.concat([numeric,dummies],axis=1)

In [167]:
X = all_in_one.loc[:,all_in_one.columns!='arr_delay']
y = all_in_one[['air_time']]

In [169]:
X_test,X_train,y_test,y_train = train_test_split(X,y,test_size=0.3,random_state=123)

In [171]:
#normalize test train
min_max = MinMaxScaler()

cols_train = X_train.columns
values_train = X_train.values
values_train_norm = min_max.fit_transform(values_train)
X_train = pd.DataFrame(values_train_norm,columns=cols_train)

cols_test = X_test.columns
values_test = X_test.values
values_test_norm = min_max.fit_transform(values_test)
X_test = pd.DataFrame(values_test_norm,columns=cols_test)

In [179]:
xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 10, alpha = 10, n_estimators = 50)

xg_reg.fit(X_train,y_train)
filname = 'XGBRegressor'


In [181]:
y_pred = xg_reg.predict(X_test)

In [184]:
y_pred

array([159.12666 , 107.743355,  96.190605, ..., 116.23766 , 132.98116 ,
        34.34118 ], dtype=float32)

In [197]:
rmse = np.sqrt(mean_squared_error(y_test,y_pred))
r2 = r2_score(y_test,y_pred)

print('rmse: ',rmse)
print('r2_score:', r2)

rmse:  11.148654921132634
r2_score: 0.976721787425578
